In [4]:
# === 1. IMPORT LIBRARIES ===
import pandas as pd
import numpy as np

# === 2. READ AND PREPARE DATA ===
# Load dataset
df = pd.read_csv("/content/dataset.csv")

# Combine date and time into a single timestamp column
df["timestamp"] = pd.to_datetime(df["LastUpdatedDate"] + " " + df["LastUpdatedTime"],
                                  format="%d-%m-%Y %H:%M:%S")

# Rename columns for simplicity
df = df.rename(columns={
    "SystemCodeNumber": "lot_id",
    "Capacity": "capacity",
    "Occupancy": "occupancy",
    "QueueLength": "queue_length",
    "TrafficConditionNearby": "traffic",
    "IsSpecialDay": "is_special",
    "VehicleType": "vehicle_type"
})

# Sort data chronologically
df = df.sort_values(by=["lot_id", "timestamp"]).reset_index(drop=True)

# === 3. ENCODE CATEGORICAL COLUMNS ===
# Convert traffic to numeric levels
traffic_map = {"low": 0, "medium": 1, "high": 2}
df["traffic_level"] = df["traffic"].map(traffic_map)

# Assign weights to vehicle types
vehicle_weight = {"bike": 0.5, "car": 1.0, "truck": 1.5}
df["vehicle_weight"] = df["vehicle_type"].map(vehicle_weight)

# === 4. DEFINE BASE VALUES AND MODELS ===
BASE_PRICE = 10.0     # Fixed base price
ALPHA = 5.0           # Coefficient for Model 1 (linear)
LAMBDA = 1.0          # Coefficient for Model 2 (scaling)

# Model 1 – Linear price increase based on utilization
def model_1(prev_price, occupancy, capacity):
    utilization = occupancy / capacity
    return prev_price + ALPHA * utilization

# Model 2 – Price based on de
def calculate_demand(row):
    utilization = row["occupancy"] / row["capacity"]
    queue = row["queue_length"]
    traffic = row["traffic_level"]
    special = row["is_special"]
    vtype = row["vehicle_weight"]

    # Using given weights (can be tuned later)
    demand = (1.0 * utilization +
              0.8 * queue -
              0.5 * traffic +
              1.2 * special +
              0.6 * vtype)
    return demand

def model_2(demand):
    # Convert raw demand to 0–1 range using sigmoid function
    demand_normalized = 1 / (1 + np.exp(-demand))
    multiplier = 1 + LAMBDA * demand_normalized
    # Clamp multiplier to be between 0.5 and 2.0
    multiplier = min(2.0, max(0.5, multiplier))
    return BASE_PRICE * multiplier

# === 5. SIMULATE PRICES OVER TIME ===
results = []

for lot_id, group in df.groupby("lot_id"):
    group = group.sort_values("timestamp")
    prev_price = BASE_PRICE

    for _, row in group.iterrows():
        # Choose the model you want to test:

        # ------ Model 1 ------
        # next_price = model_1(prev_price, row["occupancy"], row["capacity"])

        # ------ Model 2 ------
        demand = calculate_demand(row)
        next_price = model_2(demand)